In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import h5py
import glob

In [2]:
matplotlib.rc('xtick', labelsize=14)
matplotlib.rc('ytick', labelsize=14)
matplotlib.rcParams['font.size']=14

In [3]:
def calculate_fourier_conversion(Boxsize):
    '''
    Takes the Boxsize in kpc and produces the conversion factors
    from Fourier units to physical units. For use with genPK output.
    '''
    Boxsize = Boxsize / 1000 # convert to Mpc

    k_conv = 2*np.pi / Boxsize
    p_conv = (Boxsize / (2 * np.pi))**3
    
    return k_conv, p_conv

In [4]:
def get_genPK_data(fpath, k_conv, p_conv):
    
    genPK = np.loadtxt(fpath)
    
    bins = genPK[:, 0]
    pk = genPK[:, 1]
    
    dk = pk * (2 * np.pi)**3 * (4 * np.pi) * bins**3
    
    bins = genPK[:, 0] * k_conv
    
    pk = genPK[:, 1] * p_conv
    
    return bins, pk, dk

In [5]:
def wnumber2wlength(k):
    return 1/k
def wlength2wnumber(l):
    return 1/l

In [6]:
def load_data(pk_path, snap_path):
    
    snap = h5py.File(snap_path, 'r')
    
    redshift = np.round(float(snap['Header'].attrs['Redshift']), decimals=1)
    boxsize = np.round(float(snap['Header'].attrs['BoxSize']), decimals=1)
    
    k_conv, p_conv = calculate_fourier_conversion(boxsize)
    
    bins, pk, dk = get_genPK_data(pk_path, k_conv, p_conv)
    
    return np.array([bins, pk, dk]), redshift, boxsize

In [7]:
# separates data by redshift
def sort_data(pk_paths, snap_paths):
    
    datas = []
    redshifts = []
    boxsizes = []
    
    for p_path, s_path in zip(pk_paths, snap_paths):
    
        data, redshift, boxsize = load_data(p_path, s_path)
        
        datas.append(data)
        redshifts.append(redshift)
        boxsizes.append(boxsize)
    
    datas = np.array(datas)
    redshifts = np.array(redshifts)
    boxsizes = np.array(boxsizes)
    
    # doing this to remove duplicates; prevents looping too many times
    zs = list(set(redshifts))
    
    s = []

    
    for redshift in zs:
        
        truth = ( redshift == redshifts )
        
        s.append( [datas[truth], redshifts[truth], boxsizes[truth]] )
    
    
    '''
    reference for s:
    
    s contains many indices because it is a list of arrays of arrays of arrays...
    
    the first index, s[i], changes which redshift the data corresponds to
    e.g., s[0] will give all data corresponding to z=8.96
    
    the second index, s[i][j], selects what info you're looking at:
    s[i][0] is the arrays of power spectrum data
    s[i][1] is the array of redshifts
    s[i][2] is the array of boxsizes
    
    the third index, s[i][j][k], selects which particular dataset you're looking at
    e.g., s[0][0][1] will give the first 2cDM power spectrum data at z=8.96
    
    by convention, the s[i][j][0] will correspond to the CDM simulation
    
    this itself, s[i][0][k], will contain 3 arrays:
    
    s[i][j][0] = [k_bins, raw_pk, dk]
    
    this will be a mouthful to process, but should make automating plotting easier
    
    ''' 
    
    return s

In [17]:
def make_plots(pk_paths, snap_paths, out_path, labeler=''):
    
    s = sort_data(pk_paths, snap_paths)
    
    # this for each loop will pull out different datsets per redshift
    for dset in s:
        
        redshift = dset[1][0] # safe to assume this since they should be all the same redshift
                              # could add a check later
        
        pk_data = dset[0] # 0th element is the data for all snapshots
        
        bins = pk_data[:, 0]
        pks = pk_data[:, 1]
        dks = pk_data[:, 2]
                
        cdm_bin = bins[0]
        cdm_pk = pks[0]
        cdm_dk = dks[0]
        
        # raw plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bin, cdm_pk, label='CDM', color='k', linewidth=2, linestyle='--')
        
        for i in range(1, len(bins)):
            ax.plot(bins[i], pks[i])
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Power_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        # dk plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bin, cdm_dk, label='CDM', color='k', linewidth=2, linestyle='--')
        
        for i in range(1, len(bins)):
            ax.plot(bins[i], dks[i])
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$\Delta^{2}(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Dimensionless Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Dimless_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        """# spectrum ratio plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        for i in range(1, len(bins)):
            ax.plot(bins[i], pks[i] / cdm_pk)
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P_{2cDM}/P_{CDM}$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('Ratio of DM Power Spectra z = {}'.format(redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_Spectrum_Ratio_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()"""
 
        ############
        ##AVERAGES##
        ############
        
        avg_pk = np.mean(pks[1:], axis=0)
        std_pk = np.std(pks[1:], axis=0)
        avg_dk = np.mean(dks[1:], axis=0)
        std_dk = np.std(dks[1:], axis=0)
        
        #print(std_dk)
        
        # raw plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bin, cdm_pk, label='CDM', color='k', linewidth=2, linestyle='--')
        
        ax.plot(bins[0], avg_pk, label='Average 2cDM', color='r')
        ax.fill_between(bins[0], avg_pk - std_pk, avg_pk + std_pk, color='b', alpha=0.5)
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Power_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        
        # dk plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(cdm_bin, cdm_dk, label='CDM', color='k', linewidth=2, linestyle='--')
        
        ax.plot(bins[0], avg_dk, label='Average 2cDM', color='r')
        ax.fill_between(bins[0], avg_dk - std_dk, avg_dk + std_dk, color='b', alpha=0.5)
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$\Delta^{2}(k)$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('{} Dimensionless Power Spectrum z = {}'.format(labeler, redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Dimless_Spectrum_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()
        
        

        """# spectrum ratio plots
        
        fig, ax = plt.subplots(figsize=[12,12])
        
        ax.plot(bins[0], avg_pk / cdm_pk, label='Average 2cDM')
        
        ax.set_xlabel('k (h/Mpc)')
        ax.set_ylabel('$P_{2cDM}/P_{CDM}$')
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.plot([],[], label='z = {}'.format(redshift), alpha=0)
        ax.plot([],[], label='(0, 0) $\sigma = 0.1$', alpha=0)
        ax.plot([],[], label='$\Delta m/m$ = 5.56e-8', alpha=0)
        ax.set_xlim(0, 10**3)

        sax = ax.secondary_xaxis('top', functions=(wnumber2wlength, wlength2wnumber))
        sax.set_xlabel('$\lambda$ (Mpc/h)')
        plt.title('Ratio of DM Power Spectra z = {}'.format(redshift))
        plt.legend()
        plt.grid(True, which="both", ls="-")
        fname = '{}_AVG_Spectrum_Ratio_z={}.png'.format(labeler, redshift)
        plt.savefig(out_path+fname, dpi=150)
        #plt.show()
        plt.close()"""
        

In [9]:
l0 = ['/media/ryan/Elements/run_CDM_L3N128/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l1 = ['/media/ryan/Elements/run_2cDM_L3N128/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l2 = ['/media/ryan/Elements/run_2cDM_L3N128_var2/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l3 = ['/media/ryan/Elements/run_2cDM_L3N128_var3/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l4 = ['/media/ryan/Elements/run_2cDM_L3N128_var4/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l5 = ['/media/ryan/Elements/run_2cDM_L3N128_var5/snap_00{}.hdf5'.format(i) for i in range(1,8)]
l6 = ['/media/ryan/Elements/run_2cDM_L3N128_var6/snap_00{}.hdf5'.format(i) for i in range(1,8)]

spaths = l0 + l1 + l2 + l3 + l4 + l5 + l6

In [10]:
cdm_str = 'PK-DM-snap_00{}_CDM.hdf5'
tcdm_str = 'PK-DM-snap_00{}_2cDM.hdf5'

d0 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+cdm_str.format(i) for i in range(1,8)]
d1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+tcdm_str.format(i) for i in range(1,8)]
d2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+tcdm_str.format(i) for i in range(1,8)]
d3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+tcdm_str.format(i) for i in range(1,8)]
d4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+tcdm_str.format(i) for i in range(1,8)]
d5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+tcdm_str.format(i) for i in range(1,8)]
d6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+tcdm_str.format(i) for i in range(1,8)]

ppaths = d0 + d1 + d2 + d3 + d4 + d5 + d6

In [18]:
outpath = '/home/ryan/Downloads/plots_L3N128_variance/'
make_plots(ppaths, spaths, outpath, labeler='DM')

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero

In [19]:
cdm_str = 'PK-by-snap_00{}_CDM.hdf5'
tcdm_str = 'PK-by-snap_00{}_2cDM.hdf5'

b0 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+cdm_str.format(i) for i in range(1,8)]
b1 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128/'+tcdm_str.format(i) for i in range(1,8)]
b2 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var2/'+tcdm_str.format(i) for i in range(1,8)]
b3 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var3/'+tcdm_str.format(i) for i in range(1,8)]
b4 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var4/'+tcdm_str.format(i) for i in range(1,8)]
b5 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var5/'+tcdm_str.format(i) for i in range(1,8)]
b6 = ['/home/ryan/projects/Medvedev/dm_sims/genPK/run_L3N128_var6/'+tcdm_str.format(i) for i in range(1,8)]

bpaths = b0 + b1 + b2 + b3 + b4 + b5 + b6

In [20]:
outpath = '/home/ryan/Downloads/plots_L3N128_variance/'
make_plots(bpaths, spaths, outpath, labeler='BY')

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero

/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:68: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
/home/ryan/anaconda3/lib/python3.7/site-packages/numpy/ma/core.py:6759: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/ryan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero